In [18]:
import numpy as np
import sys
import itertools
from matplotlib import pyplot as plt
from helper_funcs import save_stacked, stack_combinations
from copy import deepcopy
base_path = '/home3/ebrahim2/' # replace with your base path

In [19]:
sys.path.append('beyond-brainscore/analyze_results/figures_code')
from trained_untrained_results_funcs import find_best_layer, loop_through_datasets

In [21]:
best_gpt2xl_layer_dict = np.load(f"{base_path}beyond-brainscore/analyze_results/figures_code/best_layer_sigma_info/best_gpt2xl_layer.npz")
best_pereira_layers = np.load(f"{base_path}beyond-brainscore/analyze_results/figures_code/best_layer_sigma_info/best_layer_other_pereira.npz")
best_roberta_layers = np.load(f"{base_path}beyond-brainscore/analyze_results/figures_code/best_layer_sigma_info/roberta-large.npz")
best_llama_layers = np.load(f"{base_path}beyond-brainscore/analyze_results/figures_code/best_layer_sigma_info/Llama-3.2-3B-Instruct.npz")
best_rwkv_layers = np.load(f"{base_path}beyond-brainscore/analyze_results/figures_code/best_layer_sigma_info/rwkv-4-3b-pile.npz")

In [23]:
pereira_WN_model = np.load("/data/LLMs/data_processed/pereira/acts/X_word-num.npz")['layer1']
pereira_glove_sp = np.load("/data/LLMs/data_processed/pereira/acts/X_glove-sp.npz")['layer_0']
best_pos = {'384': 3.4, '243': 2.9}
pereira_all_simple_models = {'WN': pereira_WN_model, 'glove': pereira_glove_sp}

fedorenko_WP_model = np.load("/data/LLMs/data_processed/fedorenko/acts/X_pos.npz")['layer_4.7']
fedorenko_WP_model_pearsonr = np.load("/data/LLMs/data_processed/fedorenko/acts/X_pos.npz")['layer_4.3']

fed_glove_sp = np.load("/data/LLMs/data_processed/fedorenko/acts/X_glove-sp.npz")['layer_0']
fedorenko_all_simple_models = {'WP': fedorenko_WP_model, 'glove': fed_glove_sp}

blank_pos_model = np.load("/data/LLMs/data_processed/blank/acts/X_pos.npz")['layer_11'] 
blank_pos_model_pearsonr = np.load("/data/LLMs/data_processed/blank/acts/X_pos.npz")['layer_12'] 

blank_WN_model = np.load("/data/LLMs/data_processed/blank/acts/X_WN.npz")['layer1'] 
blank_glove_sp = np.load("/data/LLMs/data_processed/blank/acts/X_glove-sp.npz")['layer_0']
blank_all_simple_models = {'pos': blank_pos_model, 'WN': blank_WN_model, 'glove': blank_glove_sp}

all_simple_models = {'pereira': pereira_all_simple_models, 'fedorenko': fedorenko_all_simple_models, 'blank': blank_all_simple_models}

In [4]:
save_stacked('', None, all_simple_models_save=deepcopy(all_simple_models), best_pos=best_pos, exclude_non_LLM=False)
save_stacked('roberta-large', best_roberta_layers, all_simple_models_save=deepcopy(all_simple_models), best_pos=best_pos)
save_stacked('Llama-3.2-3B-Instruct', best_llama_layers, all_simple_models_save=deepcopy(all_simple_models), best_pos=best_pos)
save_stacked('rwkv-4-3b-pile', best_rwkv_layers, all_simple_models_save=deepcopy(all_simple_models), best_pos=best_pos)
save_stacked('gpt2-xl', best_gpt2xl_layer_dict, all_simple_models_save=deepcopy(all_simple_models), best_pos=best_pos)

WN [1]
pos [4]
glove [300]
WN+pos [5]
WN+glove [1, 300]
pos+glove [4, 300]
WN+pos+glove [5, 300]
WN [1]
pos [4]
glove [300]
WN+pos [5]
WN+glove [1, 300]
pos+glove [4, 300]
WN+pos+glove [5, 300]
WN [1]
pos [4]
glove [300]
WN+pos [5]
WN+glove [1, 300]
pos+glove [4, 300]
WN+pos+glove [5, 300]
WN [1]
pos [4]
glove [300]
WN+pos [5]
WN+glove [1, 300]
pos+glove [4, 300]
WN+pos+glove [5, 300]
WN [1]
pos [4]
glove [300]
WN+pos [5]
WN+glove [1, 300]
pos+glove [4, 300]
WN+pos+glove [5, 300]
WN [1]
pos [4]
glove [300]
WN+pos [5]
WN+glove [1, 300]
pos+glove [4, 300]
WN+pos+glove [5, 300]
WP [9]
WP [9]
WP [9]
WN [1]
pos [10]
WN+pos [11]
WN [1]
pos [10]
WN+pos [11]
WN [1]
pos [10]
WN+pos [11]


# Stacking for pearsonr

In [15]:
PWR_pereira_384 = np.load("/data/LLMs/data_processed/pereira/acts/X_positional_WN_smooth.npz")['layer_1.2']
PWR_pereira_243 = np.load("/data/LLMs/data_processed/pereira/acts/X_positional_WN_smooth.npz")['layer_0.5']

np.savez('/data/LLMs/data_processed/pereira/acts/X_glove-sp_PWR_384', **{'layer_1': np.hstack((pereira_glove_sp, PWR_pereira_384))})
np.savez('/data/LLMs/data_processed/pereira/acts/X_glove-sp_PWR_243', **{'layer_1': np.hstack((pereira_glove_sp, PWR_pereira_243))})

np.savez('/data/LLMs/data_processed/fedorenko/acts/X_glove-sp_PWR', **{'layer_1': np.hstack((fed_glove_sp, fedorenko_WP_model_pearsonr))})

np.savez('/data/LLMs/data_processed/blank/acts/X_glove-sp_PWR', **{'layer_1': np.hstack((blank_glove_sp, blank_pos_model_pearsonr))})


In [5]:
# stack syntax
for fe in ['','-mp', '-sp']:
    np.load(f'/data/LLMs/data_processed/pereira/acts/X_gpt2xl-syntax{fe}.npz')